In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from datasets import Dataset
import joblib

In [ ]:
df=pd.read_csv("chats.csv")
print(df.head())

In [ ]:
df=df.dropna()
df['Message']=df['Message'].astype(str)

In [ ]:
df['context']=""
for i in range(1,len(df)):
    df.loc[i,'context']=" ".join(df.loc[:i-1,'Message'].tail(3).values)

In [ ]:
train_texts,test_texts=train_test_split(df,test_size=0.1,random_state=42)
train_df=pd.DataFrame(train_texts)
test_df=pd.DataFrame(test_texts)

In [ ]:
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
def tokenize(batch):
    tokenized_inputs = tokenizer([c + " " + m for c, m in zip(batch['context'], batch['Message'])],
                                 padding='max_length', truncation=True, max_length=128)
    return tokenized_inputs

In [ ]:
train_dataset=Dataset.from_pandas(train_df).map(tokenize,batched=True)
test_dataset=Dataset.from_pandas(test_df).map(tokenize,batched=True)
train_dataset.set_format(type='torch',columns=['input_ids','attention_mask'])
test_dataset.set_format(type='torch',columns=['input_ids','attention_mask'])

In [ ]:
model=GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

In [ ]:
training_args=TrainingArguments(output_dir="./results",num_train_epochs=1,per_device_train_batch_size=2,per_device_eval_batch_size=2,warmup_steps=10,logging_dir="./logs",save_strategy="no")
trainer=Trainer(model=model,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset)
trainer.train()

In [ ]:
def generate_reply(context):
    inputs=tokenizer.encode(context,return_tensors='pt')
    outputs=model.generate(inputs,max_length=50,num_return_sequences=1,pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0],skip_special_tokens=True)

In [ ]:
sample=df.sample(3)
for _,row in sample.iterrows():
    c=row['context']
    print("Context:",c)
    print("Generated:",generate_reply(c))
    print()

In [ ]:
refs=[row['Message'].split() for _,row in test_df.head(10).iterrows()]
hyps=[generate_reply(row['context']).split() for _,row in test_df.head(10).iterrows()]
bleu_scores=[sentence_bleu([r],h) for r,h in zip(refs,hyps)]
print("Average BLEU:",np.mean(bleu_scores))

In [ ]:
joblib.dump(model,"Model.joblib")
print("Model saved.")